In [12]:
import numpy
import scipy.special
import matplotlib.pyplot
%matplotlib inline
#test

class neuralNetwork:
    
    #initialize neuralNetwork
    def __init__(self, inputNodes, hiddenNodes, outputNodes, learningRate):
        
        #initialize size of neuralNetwork
        self.iNodes = inputNodes
        self.hNodes = hiddenNodes
        self.oNodes = outputNodes
        
        #link weight matrices, Wih and Who
        #weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        #W1.1 W2.1
        #W1.2 W2.2 etc.
        #weights from inputNodes to hiddenNodes
        self.Wih = numpy.random.normal(0.0, pow(self.iNodes, -0.5), (self.hNodes, self.oNodes))
        #weights from hiddenNodes to outputNodes
        self.Who = numpy.random.normal(0.0, pow(self.hNodes, -0.5), (self.oNodes, self.hNodes))
        
        #initialize learning rate for neuralNetwork
        self.lr = learningRate
        
        #activation functions aka Sigmoid fucntion
        self.activation_function = lambda x: scipy.special.expit(x)
        pass
    
    #train neuralNetwork
    def train(self, input_list, target_list):
        
        #convert input list to 2d array
        inputs = numpy.array(input_list, ndmin=2).T
        #convert target list to 2d array
        targets = numpy.array(target_list, ndmin=2).T
        
        #calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.Wih, inputs)
        #calucalte the signals emerging from the hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        #calculate signals into final output layer
        final_inputs = numpy.dot(self.Who, hidden_outputs)
        #calculate the signals emerging from the output layer
        final_outputs = self.activation_function(final_inputs)
        
        #error is (target - actual)
        output_errors = targets - final_outputs
        
        #hidden layer error is the output errors, split by the weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.Who.T, output_errors)
        
        #update the weights for the links between the hidden and output layers
        self.Who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        #update the weights for the links between the input and hidden layers
        self.Wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        pass
    
    #query neuralNetwork
    def query(self, inputList):
        
        #convert inputs list to 2d array
        inputs = numpy.array(inputList, ndmin = 2).T
        
        #calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.Wih, inputs)
        #calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        #calculate signals into final output layer
        final_inputs = numpy.dot(self.Who, hidden_outputs)
        #calculate the signals emerging from the final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [18]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.1

# create instance of neural network
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [19]:
training_data_file = open("mnist_datasets/mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [20]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 5

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

ValueError: shapes (200,10) and (784,1) not aligned: 10 (dim 1) != 784 (dim 0)